<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_001_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Validación cruzada</font>**

<p align="justify">
En los Colabs anteriores, se ven algunos conceptos relacionados con la evaluación de modelos predictivos. Si bien esta sección podría ser un poco redundante, tenemos la intención de profundizar el tema de validación cruzada.
<br><br>
Antes de comenzar, detengámonos en las razones para tener un conjunto de datos de entrenamiento y otro conjunto de datos de prueba. Pero antes, vamos a ver la limitación de usar todo el conjunto de datos, sin excluir ninguna muestra.
<br><br>
Para ilustrar esos los diferentes conceptos, utilizaremos el conjunto de datos de vivienda de California, pero previo a comenzar, vamos a definir el tema de este Colab, la validación cruzada.

 ## **<font color="DeepPink">Definición y conceptos de la validación cruzada</font>**

<p align="justify">
La validación cruzada, también conocida como <code>cross-validation</code> en inglés, es una técnica utilizada en el aprendizaje automático para evaluar el rendimiento de un modelo estadístico o algoritmo de predicción.
<br><br>
En <code>Scikit-learn</code>, la biblioteca seleccionada para desarrollar modelos de aprendizaje automático, la validación cruzada se implementa mediante la clase <code>cross_val_score</code> del módulo <code>model_selection</code>. Esta clase permite realizar la validación cruzada de manera sencilla y eficiente.
<br><br>
La validación cruzada divide el conjunto de datos en $k$ partes o pliegues denominados (folds) en ingles, los que son casi de igual tamaño. Luego, se entrena y evalúa el modelo $k$ veces, utilizando cada vez un solo pliegue diferente como conjunto de prueba y los restantes pliegues como conjunto de entrenamiento. De esta manera, se obtiene una medida de rendimiento promedio sobre los $k$ pliegues, lo que proporciona una estimación más fiable del rendimiento del modelo.
<br><br>
Por ese motivo, el proceso de validación cruzada permite evaluar cómo se generaliza el modelo a nuevos datos y ayuda a evitar problemas como el sobreajuste (overfitting) o el sesgo (bias). Además, proporciona una estimación más precisa del rendimiento del modelo que se obtendría al evaluarlo solo en un conjunto de prueba único.
<br><br>
En resumen, la validación cruzada en <code>Scikit-learn</code> es una técnica para evaluar el rendimiento de los modelos de aprendizaje automático al dividir los datos en pliegues, entrenar y evaluar el modelo en cada pliegue, y luego obtener una medida de rendimiento promedio. Esto ayuda a tener una estimación más fiable del rendimiento y a evitar problemas como el sobreajuste.

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.express as px

 ## **<font color="DeepPink">Carga y análisis de conjunto de datos</font>**

In [3]:
from sklearn.datasets import fetch_california_housing

In [4]:
housing = fetch_california_housing(as_frame=True) #as_frame: como Pandas DataFrame

In [5]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

<p align="justify">
👀 En este conjunto de datos, el objetivo es predecir el valor promedio de las casas en un área de California. Las características recopiladas se basan en información inmobiliaria y geográfica general de estas propiedades. Como el valor promedio a predecir es el precio medio (variable numérica continua), utilizaremos entonces un modelo predictivo específico de regresión. 

<p align="justify">
🛑 Vemos que el objeto <code>housing</code> es un objeto de <code>Scikit-learn</code> y tiene sus propios métodos y atributos, como por ejemplo <code>data</code> que lo muestra o transforma en un <code>DataFrame</code>

In [6]:
type(housing)

sklearn.utils._bunch.Bunch

In [7]:
housing.data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [8]:
housing.target

0        4.526
1        3.585
2        3.521
3        3.413
4        3.422
         ...  
20635    0.781
20636    0.771
20637    0.923
20638    0.847
20639    0.894
Name: MedHouseVal, Length: 20640, dtype: float64

<p align="justify">
👀 Ahora si, dividimos todos los datos del conjunto de datos <code>housing</code>, en nuestra variable objetivo y las variables explicativas. En el caso de la variable objetivo hacemos una transformación de los datos para expresar los valores a cientos de miles de dólares. 

In [9]:
y = housing.target * 100000 #transformamos el precio a cientos de miles de dólares.
X = housing.data

<p align="justify">
👀 Visualizamos variable objetivo. 

In [10]:
y.head()

0    452600.0
1    358500.0
2    352100.0
3    341300.0
4    342200.0
Name: MedHouseVal, dtype: float64

<p align="justify">
👀 Visualizamos variables explicativas.

In [11]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


<p align="justify">
👀 En este caso, no vamos a utilizar las variables <code>Latitude</code> y <code>Longitude</code>, pero sería interesante tenerlas en cuenta tambien como explicativas del precio de la propiedad. 

In [12]:
X.drop(columns=["Latitude", "Longitude"])

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467
...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209


 # **<font color="DeepPink">Error de entrenamiento versus Error de prueba</font>**


 ## **<font color="DeepPink">Creación y ajuste del modelo</font>**

Para resolver el problema de regresión, utilizaremos un [Árbol de Regresión](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

<p align="justify">
Un árbol de decisión en <code>Scikit-learn</code> es un algoritmo de aprendizaje automático supervisado utilizado para resolver problemas de clasificación y regresión. Se basa en la idea de construir un modelo en forma de árbol que toma decisiones secuenciales basadas en características o atributos de los datos de entrada, o variables explicativas.
<br><br>
En <code>Scikit-learn</code>, los árboles de decisión se implementan a través de la clase <code>DecisionTreeClassifier</code> para problemas de clasificación y la clase <code>DecisionTreeRegressor</code> para problemas de regresión. 

In [13]:
from sklearn.tree import DecisionTreeRegressor

In [14]:
model = DecisionTreeRegressor(random_state=0) #se usa random_state para controlar la aleatoriedad del estimador.
model.fit(X, y)

DecisionTreeRegressor(random_state=0)

<p align="justify">
👀 Entonces la creación del árbol se hace con la instancia de la clase  <code>DecisionTreeRegressor</code>, y se ajusta al conjunto de datos  utilizando el método <code>fit()</code>. Durante este proceso, el árbol se construye de forma recursiva dividiendo los datos en función de las características para maximizar la pureza en los nodos hoja (clasificación) o reducir el error cuadrático medio (regresión), y en nuestro caso será la reducción del error cuadrático medio.

 ## **<font color="DeepPink">Evaluación del modelo</font>**

<p align="justify">
Después de entrenar el regresor, nos gustaría conocer su rendimiento potencial una vez implementado en producción. Para este propósito, usamos el Error Absoluto Medio (MAE), que nos da un error en la unidad original, es decir, en $.

In [15]:
from sklearn.metrics import mean_absolute_error

In [16]:
y_predicted = model.predict(X)
score = mean_absolute_error(y, y_predicted)
print("")
print(f"En promedio, nuestro modelo genera un error de $ {score:.2f}")


En promedio, nuestro modelo genera un error de $ 0.00


<p align="justify">
Vemos que obtenemos una predicción perfecta sin errores de los precios de las propiedades. Esto es demasiado optimista y cuando sucede, casi siempre revela un problema metodológico. 
<br><br>
👀 Entrenamos y predijimos con el mismo conjunto de datos. 
<br><br>
En nuestro árbol de decisión, cada muestra en el conjunto de datos se almacena en un nodo hoja. Por lo tanto, nuestro árbol de decisiones memorizó completamente el conjunto de datos proporcionado durante el ajuste del modelo <code>fit()</code> y por ese motivo, no cometió ningún error al predecir los valores de los precios de las propiedades.



## **<font color="DeepPink">División de datos de prueba y entrenamiento</font>**

<p align="justify">
🛑 El error calculado anteriormente se denomina <b>error empírico</b> o <b>error de entrenamiento</b>.
<br><br>
Entrenamos un modelo predictivo para minimizar el error de entrenamiento, pero nuestro objetivo es minimizar el error en los datos que no se han visto durante el entrenamiento del modelo.
<br><br>
Este error también se denomina <b>error de generalización</b> o <b>error de prueba "verdadero"</b>.
<br><br>
Así, la evaluación más básica implica:

* Dividir el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba;
* Ajustar el modelo en el conjunto de entrenamiento;
* Estimar el error de entrenamiento en el conjunto de entrenamiento;
* Estimar el error de prueba en el conjunto de prueba.

<br>
<p align="justify">
👀 Así que dividamos nuestro conjunto de datos, para ello <code>Scikit-learn</code> proporciona una función llamada <code>train_test_split</code> del módulo <code>model_selection</code> que se utiliza para realizar esta división. La función <code>train_test_split</code> toma como entrada el conjunto de datos completo y lo divide en dos (o más) conjuntos: uno para entrenamiento y otro para evaluación.
<br><br>
La división del conjunto de datos se realiza de manera aleatoria, asegurando que los datos se mezclen para evitar cualquier sesgo o patrón específico en los conjuntos de entrenamiento y evaluación. Por lo general, la división se realiza asignando un porcentaje específico de los datos al conjunto de evaluación (también conocido como conjunto de prueba) y el restante al conjunto de entrenamiento.
<br><br>
La función <code>train_test_split</code> también permite realizar divisiones estratificadas, lo que significa que mantiene la proporción de clases en ambos conjuntos. Esto es útil en problemas de clasificación, donde se desea que la distribución de clases sea representativa en el conjunto de entrenamiento y evaluación.

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [19]:
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

<p align="justify">
<code>X_train</code> y <code>y_train</code> representan los conjuntos de entrenamiento, mientras que <code>X_test</code> y <code>y_test</code> representan los conjuntos de evaluación. El argumento <code>random_state</code> se utiliza para fijar la semilla aleatoria y asegurar que la división sea reproducible, es decir, independientemente de quien ejecute el script, siempre se obtenga el mismo resultado.
<br><br>
La división del conjunto de datos en <code>Scikit-learn</code> es esencial para evaluar el rendimiento de los modelos de aprendizaje automático de manera objetiva y para evitar el sobreajuste al utilizar los mismos datos tanto para entrenamiento como para evaluación (test).

 ## **<font color="DeepPink">Evaluación del modelo</font>**

👀 Error de entrenamiento:

In [20]:
y_predicted = model.predict(X_train)
score = mean_absolute_error(y_train, y_predicted)
print("")
print(f"El error de entrenamiento del modelo es $ {score:.2f}")


El error de entrenamiento del modelo es $ 0.00


👀 Error de prueba:

In [21]:
y_predicted = model.predict(X_test)
score = mean_absolute_error(y_test, y_predicted)
print("")
print(f"El error de prueba del modelo es $ {score:.2f}")


El error de prueba del modelo es $ 47378.08


<p align="justify">
✅ Este error de prueba es en realidad lo que esperaríamos de nuestro modelo si se usara en un entorno de producción.

# **<font color="DeepPink">Estabilidad de las estimaciones de Validación Cruzada</font>**

<p align="justify">
Al hacer una sola división de prueba y entrenamiento, no damos ninguna indicación con respecto a la robustez de la evaluación de nuestro modelo predictivo: en particular, si el conjunto de prueba es pequeño, esta estimación del error de prueba será inestable y no reflejaría la "tasa de error real" que habríamos observado con el mismo modelo en una cantidad ilimitada de datos de prueba.
<br><br>
Por ejemplo, podríamos haber tenido suerte cuando hicimos nuestra división aleatoria de nuestro conjunto de datos limitado y aislar algunos de los casos más fáciles de predecir en el conjunto de prueba por casualidad: la estimación del error de prueba sería demasiado optimista, en este caso.
<br><br>
Entonces la <b>validación cruzada</b> permite estimar la robustez de un modelo predictivo por repitiendo el procedimiento de división. Dará varios errores de entrenamientos y pruebas. y, por tanto, alguna <b>estimación de la variabilidad de la generalización del modelo rendimiento</b>.

Hay [diferentes estrategias de validación cruzada](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators), 

<p align="justify">
Por ahora nos vamos a centrar en uno llamado <code>shuffle-split</code>. En cada iteración de esta estrategia nosotros vamos a:


- Mezclar aleatoriamente el orden de las muestras del conjunto de datos completo;
- Dividir el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba;
- Entrenar un nuevo modelo con el conjunto de entrenamiento;
- Evaluar el error de prueba con el conjunto de prueba.

<p align="justify">
Repetimos este procedimiento <code>n_splits</code> veces. Hay que tener en cuenta que el costo computacional aumenta con <code>n_splits</code>.

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Validacion-001.png?raw=true" width="600">
</p>


<p align="justify">
Esta figura muestra el caso particular de la estrategia de validación cruzada <code>shuffle-split</code> usando <code>n_splits = 5</code>. Para cada división de validación cruzada, el procedimiento entrena un modelo en todas las muestras rojas y evalúa la puntuación del modelo en las muestras azules.

<p align="justify">
En este caso estableceremos <code>n_splits=40</code>, lo que significa que entrenaremos $40$ modelos en total y todos ellos serán descartados: solo registramos su rendimiento de generalización en cada variante del conjunto de prueba.


Para evaluar el desempeño de nuestro modelo, debemos usar:

- [`sklearn.model_selection.cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)
con un objeto
- [`sklearn.model_selection.ShuffleSplit`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html)

In [22]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

In [23]:
cv = ShuffleSplit(n_splits=40, test_size=0.3, random_state=0)
cv_results = cross_validate(model, X, y, cv=cv, scoring="neg_mean_absolute_error")

In [24]:
cv_results

{'fit_time': array([0.17140079, 0.17506695, 0.17244935, 0.16458821, 0.17326975,
        0.17195749, 0.21093702, 0.2576735 , 0.24424458, 0.24315596,
        0.23996949, 0.24746466, 0.25013471, 0.23573136, 0.1938355 ,
        0.17082763, 0.16594172, 0.16853642, 0.1650331 , 0.16166472,
        0.16366506, 0.16486073, 0.17013931, 0.16409063, 0.16712451,
        0.16372776, 0.1623795 , 0.16369343, 0.17512631, 0.1636157 ,
        0.16321683, 0.16555786, 0.16902781, 0.1639185 , 0.1732111 ,
        0.16799116, 0.1635294 , 0.16124845, 0.1653316 , 0.1666069 ]),
 'score_time': array([0.00431395, 0.00433803, 0.0044241 , 0.00457907, 0.00425935,
        0.00435376, 0.00529051, 0.00661922, 0.00542498, 0.00555325,
        0.00517607, 0.00585175, 0.00594807, 0.00590968, 0.00425339,
        0.00439548, 0.00608563, 0.00427294, 0.00431371, 0.00418305,
        0.00422645, 0.00432181, 0.00517464, 0.00471306, 0.00624585,
        0.00428772, 0.00500441, 0.00593996, 0.00422812, 0.00431299,
        0.00420022, 

<p align="justify">
👀 Los resultados <code>cv_results</code> se almacenan en un <b>diccionario</b>. Lo convertiremos a un <b>DataFrame</b> para facilitar la visualización y manipulación de los datos.

In [25]:
cv_results = pd.DataFrame(cv_results)
cv_results.head()

,fit_time,score_time,test_score
0,0.171401,0.004314,-47137.009528
1,0.175067,0.004338,-46839.433786
2,0.172449,0.004424,-47305.854490
3,0.164588,0.004579,-44851.398417
4,0.173270,0.004259,-47072.168928


<p align="justify">
✅ Una puntuación es una métrica, y los valores más altos significan mejores resultados. 
<br><br>
✅ Por el contrario, un error es una métrica en la que valores más bajos significan mejores resultados.
<br><br> 
El parámetro <code>scoring</code> en <code>cross_validate</code> siempre espera una función que sea una puntuación.
<br><br>
👀 Para hacerlo más fácil, todas las métricas de error en <code>Scikit-learn</code>, como <code>mean_absolute_error</code>, se pueden transformar en una puntuación para usar en <code>cross_validate</code>. Para hacerlo, debe pasar una cadena de la métrica de error con una cadena neg_ adicional al principio de la puntuación del parámetro; por ejemplo, <code>scoring="neg_mean_absolute_error"</code>. En este caso se computará el negativo del error absoluto medio que equivaldría a una puntuación.

<p align="justify">
👀 Ahora obtenemos la información de tiempo para ajustar y predecir en cada iteración de la validación cruzada. Además, obtenemos la puntuación de la prueba, que corresponde al error de prueba en cada una de las divisiones de la validación cruzada.

In [26]:
cv_results["test_error"] = -cv_results["test_score"]
cv_results.test_error.head()

0    47137.009528
1    46839.433786
2    47305.854490
3    44851.398417
4    47072.168928
Name: test_error, dtype: float64

In [27]:
len(cv_results)

40

<p align="justify">
👀 Vemos que tenemos $40$ entradas en nuestro <code>DataFrame</code> porque realizamos $40$ divisiones. Por lo tanto, podemos mostrar la distribución del error de prueba y así tener una estimación de su variabilidad.

In [28]:
print("")
print(f"Media: {cv_results.test_error.mean()}")
print(f"Desvío estandar: {cv_results.test_error.std()}")


Media: 46408.783781492246
Desvío estandar: 1195.4177777280847


In [29]:
px.histogram(cv_results,
             x="test_error",
             template="gridon",
             nbins=30).update_layout(bargap=0.2)

<p align="justify">
👀 Podemos observar que el error de prueba está agrupado alrededor de los $47$ y en un rango que va desde $43$ a $50$.
<br><br>
Tenga en cuenta que la desviación estándar es mucho menor que la media, y podríamos resumir que nuestra estimación de validación cruzada del error de prueba es $46.36$ ± $1,17$ k\$.
<br><br>
Si tuviéramos que entrenar un solo modelo en el conjunto de datos, es decir sin validación cruzada, y luego tener acceso a una cantidad ilimitada de pruebas de los datos, esperaríamos que su verdadero error de prueba se acerque a esa región.
<br><br>
Si bien esta información es interesante en sí misma, debe contrastarse con la escala de la variabilidad natural de la variable objetivo en nuestro conjunto de datos.

✅ Veamos la distribución de la variable objetivo:

In [30]:
px.histogram(y, template="gridon").update_layout(bargap=0.2)

In [31]:
print("")
print(f"The standard deviation of the target is: ${y.std():.2f} ")


The standard deviation of the target is: $115395.62 


<p align="justify">
✅ La variable objetivo va desde cerca de $0$ hasta $500$ y, con una desviación estándar alrededor de $115$.
<br><br>
✅ Notamos que la estimación media del error de prueba obtenido por validación cruzada es un poco más pequeña que la escala natural de variación de la variable objetivo. Además, la desviación estándar de la estimación de validación cruzada del error de prueba es aún menor.
<br><br>
✅ Este es un buen comienzo, pero no necesariamente suficiente para decidir si el rendimiento de la generalización es bueno para que nuestra predicción sea útil en la práctica.
<br><br>
Recordamos que nuestro modelo comete, en promedio, un error de alrededor de $47$. Con esta información y mirando la distribución objetivo, tal error podría ser aceptable al predecir casas de $500$. Sin embargo, sería un problema con una casa con un valor de $50$. Por lo tanto, esto indica que nuestra métrica Error Absoluto Medio (MAE) no es ideal.
<br><br>
En lugar de usar entonces el Error Absoluto Medio (MAE), podríamos elegir una métrica relativa al valor objetivo para predecir, como por ejemplo, el error porcentual absoluto medio, que habría sido una opción mucho mejor.
<br><br>
Pero en todos los casos, un error de $47$ podría ser demasiado grande para usar automáticamente nuestro modelo para etiquetar los valores de las propiedades, sin la supervisión de un experto que conozca del negocio.

# **<font color="DeepPink">Más detalles sobre `cross_validate`</font>**

<p align="justify">
👀 Durante la validación cruzada, se entrenan y evalúan muchos modelos. De hecho, el número de elementos en cada matriz de la salida de <code>cross_validate</code> es un resultado de uno de estos procedimientos <code>fit</code> / <code>score</code>. Para hacerlo explícito, es posible recuperar estos modelos ajustados para cada una de las divisiones (folds) pasando la opción <code>return_estimator=True</code> en <code>cross_validate</code>.

In [32]:
from sklearn.model_selection import cross_val_score

In [33]:
cv_results = cross_validate(model, X, y, return_estimator=True)
cv_results

{'fit_time': array([0.21872234, 0.21384168, 0.19145846, 0.19702148, 0.18613935]),
 'score_time': array([0.00307393, 0.00314784, 0.00313091, 0.00301003, 0.00302649]),
 'estimator': [DecisionTreeRegressor(random_state=0),
  DecisionTreeRegressor(random_state=0),
  DecisionTreeRegressor(random_state=0),
  DecisionTreeRegressor(random_state=0),
  DecisionTreeRegressor(random_state=0)],
 'test_score': array([0.21214626, 0.41969181, 0.42331813, 0.19377131, 0.4145845 ])}

In [34]:
cv_results['estimator']

[DecisionTreeRegressor(random_state=0),
 DecisionTreeRegressor(random_state=0),
 DecisionTreeRegressor(random_state=0),
 DecisionTreeRegressor(random_state=0),
 DecisionTreeRegressor(random_state=0)]

<p align="justify">
👀 Los cinco regresores del árbol de decisión corresponden a los cinco árboles en los diferentes folds. Tener acceso a estos regresores es útil porque permite inspeccionar los parámetros ajustados internos de estos regresores.
<br><br>
En el caso de que solo nos interese el puntaje de la prueba, <code>Scikit-learn</code> proporciona la función <code>cross_val_score</code>. Es idéntico a llamar a la función <code>cross_validate</code> y para seleccionar <code>test_score</code> solamente.

In [35]:
scores = cross_val_score(model, X, y)
scores

array([0.21214626, 0.41969181, 0.42331813, 0.19377131, 0.4145845 ])

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un objeto de<code>Scikit-learn</code> y lo convertimos a un <code>DataFrame</code>.
<br> 
✅ Vimos la necesidad de dividir los datos en un conjunto de entrenamiento y un conjunto de prueba.
<br> 
✅ Constatamos el significado de los errores de entrenamiento y los errores de prueba. 
<br> 
✅ Definimos el marco general de la validación cruzada con la posibilidad de estudiar las variaciones del rendimiento de la generalización del modelo.


<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
